<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/ufidon/nlp/blob/main/09.mlm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
  <td>
    <a target="_blank" href="https://kaggle.com/kernels/welcome?src=https://github.com/ufidon/nlp/blob/main/09.mlm.ipynb"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" /></a>
  </td>
</table>
<br>

# Masked Language Models

📝 SALP chapter 11

- Introduction to `bidirectional transformer` encoders, focusing on the `BERT` model and its masked language modeling technique.
  - [BERT](https://huggingface.co/docs/transformers/en/model_doc/bert): Bidirectional Encoder Representations from Transformers
- Explanation of `masked language modeling`, where a word is masked in the middle of a sentence and the model predicts it based on surrounding context.
- Discussion of finetuning pretrained models by `adding a classifier for downstream tasks` like named entity tagging or natural language inference.
- Introduction to `transfer learning`, where knowledge from pretraining is applied to new tasks.
- Overview of `contextual embeddings`, which represent words differently based on their context, contrasting with earlier static embeddings like word2vec or GloVe.

## Bidirectional Transformer Encoders
- Underlie [BERT](https://huggingface.co/docs/transformers/en/model_doc/bert), [RoBERTa](https://huggingface.co/docs/transformers/en/model_doc/roberta), and [SpanBERT](https://huggingface.co/SpanBERT/spanbert-large-cased), which differ from causal (left-to-right) transformers.
- Used for `contextual token representation` while causal models for generative tasks like question answering and summarization.
- Utilize information from the entire input sequence including `future tokens`
  - useful for tasks like part-of-speech tagging, parsing, and named entity recognition.
- Focus on creating contextualized token representations using self-attention across the entire input sequence.
  - input embeddings $(𝐱_1 , ⋯, 𝐱_n )$ ↦ output embeddings $(𝐡_1 , ⋯, 𝐡_n)$
- The `masked language models` are sometimes called `encoder-only`.

### The Architecture for Bidirectional Masked Models
- **Bidirectional Attention**:  
  - Bidirectional transformers pay attention to both preceding and following tokens.
  ![causal transformer vs. bidirectional attention model](./images/mlm/bitran.png)

- **Attention Formula Adjustment**:  
  ![Matrix masks](./images/mlm/mask.png)
  - In causal transformers (a), the attention formula includes a masking step:
    - $𝐀 = \text{softmax} \left( \text{mask} \left( \dfrac{𝐐𝐊^T}{\sqrt{d_k}} \right) \right) 𝐕$
  - In bidirectional transformers (b), the mask is removed:
    - $𝐀 = \text{softmax} \left( \dfrac{𝐐𝐊^T}{\sqrt{d_k}} \right) 𝐕$
    - enabling full context access across the input sequence.

- **Tokenization of Input**:  
  - The model uses subword tokenization methods, such as WordPiece, SentencePiece or BPE, processing subword tokens instead of whole words.
  - Subwords may need to be mapped back to words for certain NLP tasks like parsing.

- **BERT Architecture (English-only)**:  
  - Vocabulary of 30,000 tokens generated with WordPiece.
  - Hidden layer dimension $d = 768$.
  - 12 transformer layers, each with 12 bidirectional multihead attention.
  - Approximately 100 million parameters.

- **[XLM-RoBERTa](https://huggingface.co/docs/transformers/en/model_doc/xlm-roberta) Architecture (Multilingual: 100 Languages)**:  
  - Vocabulary of 250,000 tokens generated with SentencePiece.
  - 24 transformer layers, each with 16 bidirectional multihead attention.
  - Hidden layer dimension of 1024.
  - Input context window of 512 tokens.
  - Approximately `550 million parameters`.

- **Model Size Comparison**:  
  - Bidirectional models like BERT and XLM-RoBERTa are significantly smaller than causal models; for example, Llama 3 has `405 billion parameters`.

## Training Bidirectional Encoders
- **New Training Scheme**: 
  - Without the causal mask, predicting the next word becomes trivial
    - Because the answer is directly available from the context
  - A new task, the `cloze task`, is used instead. 
    - This `fill-in-the-blank` method requires the model to predict missing words within a sentence based on the rest of the context.

- **Training Method**: 
  - The model learns to predict the missing elements by generating a `probability distribution` over the vocabulary for each missing spot
    - using cross-entropy loss to guide learning.

- **Denoising Objective**: 
  - The model's training involves corrupting input sequences (e.g., masking, substituting, reordering, deleting, or inserting words) and tasking the model with recovering the original text. 
  - This general approach is known as `denoising`.

### Masking Words
- **Masked Language Modeling (MLM)**:
  - MLM is the original approach to train bidirectional encoders on a large corpus of unannotated text where `a random sample of tokens` from each sentence is selected.
  - Selected tokens are modified in one of three ways:
    - Replaced with a special token `[MASK]`.
    - Replaced with a random token sampled from the vocabulary based on unigram probabilities.
    - Left unchanged.

- **Token Sampling and Replacement**:
  - In BERT, 15% of tokens are sampled for modification in each training sequence:
    - 80% of selected tokens are replaced with `[MASK]`.
    - 10% are replaced with a random token.
    - 10% remain unchanged.
  
- **Training Objective**:
  - The model predicts the original value of each modified token using a bidirectional encoder, with a cross-entropy loss guiding the parameter updates.
  - All input tokens participate in the self-attention process, but only sampled tokens contribute to training loss.

- **Probability Calculation and Loss Computation**:
  ![Masked language model training](./images/mlm/trmask.png)
  - Each modified token’s embedding vector is processed through transformer layers, generating a final output vector $𝐡^L_i$.
  - The output vector is multiplied by the unembedding layer $𝐄^T$ to produce logits $𝐮$:
    - $𝐮_i = 𝐡^L_i 𝐄^T$
  - The softmax function then turns logits into a probability distribution $𝐲$ over the vocabulary:
    - $𝐲_i = \text{softmax}(𝐮_i)$
  - The loss for a token $x_i$ is based on the probability assigned to the correct word, given the modified sequence $𝐱^{\text{mask}}$:
    - $L_{\text{MLM}}(x_i) = -\log P(x_i | 𝐡^L_i)$
  - Average loss over the sampled tokens from a sequence (or batch) forms the MLM loss:
    - $L_{\text{MLM}} = - \dfrac{1}{|M|} \sum_{i \in M} \log P(x_i | 𝐡^L_i)$

- **Training Efficiency**:
  - Only tokens in `the set $M$ (sampled tokens)` contribute to training updates, meaning only 15% of input tokens drive weight adjustments, which is relatively inefficient.

### Next Sentence Prediction
- **Focus on Sentence Relationships**:  
  - Mask-based learning primarily focuses on predicting words based on context, but many applications require understanding `relationships between sentence pairs`, such as:
    - Paraphrase detection (similar meanings),
    - Entailment (checking logical implications or contradictions),
    - Discourse coherence (ensuring sentences logically follow each other).

- **Next Sentence Prediction (NSP) Task**:  
  - To capture inter-sentence relationships, some BERT models include an NSP objective, where the model predicts if two sentences are adjacent (50% true pairs, 50% random pairs).
  - The NSP loss evaluates the model's ability to distinguish actual sentence pairs from random ones.

  ![An example of the NSP loss calculation](./images/mlm/nsp.png)

- **Input Token Modifications for NSP**:  
  - BERT introduces `[CLS]` and `[SEP]` tokens in the input:
    - `[CLS]` is prepended to the input, and `[SEP]` is placed between and after the sentence pair.
    - Segment embeddings distinguish each sentence, enhancing sentence pair differentiation.

- **NSP Head and Prediction**:  
  - The output vector $𝐡^L_{\text{CLS}}$ from the final layer for the `[CLS]` token represents the NSP result.
  - The NSP head, a learned weight matrix $𝐖_{\text{NSP}} \in \mathbb{R}^{d \times 2}$, applies to $𝐡^L_{\text{CLS}}$ to produce a two-class prediction:
    - $𝐲_i = \text{softmax}(𝐡^L_{\text{CLS}} 𝐖_{\text{NSP}})$

- **Combined Loss Function**:  
  - Cross-entropy is used to compute the NSP loss for each sentence pair
  - In BERT, the final training loss combines both MLM and NSP objectives.

### Training Schemes
- **Training Datasets and Scale**:
  - Early transformer models, like BERT, were trained on 3.3 billion words from English Wikipedia and BooksCorpus.
  - Modern masked language models use larger, filtered datasets from web text, augmented with higher-quality sources like Wikipedia.
  - Multilingual model XLM-R was trained on 300 billion tokens across 100 languages from the web via [Common Crawl](https://commoncrawl.org/).

- **Training Setup for BERT**:
  - BERT models sample pairs of text segments from the corpus using the NSP objective (50/50 real or random pairs) with a 512-token limit.
  - Tokens within these pairs are masked according to MLM, and the combined MLM and NSP losses are used in training, with approximately 40 epochs needed for convergence.

- **Training Modifications in Models like RoBERTa**:
  - RoBERTa omits the NSP objective, instead using contiguous sentences as input and padding until 512 tokens are reached.
  - This approach typically uses large batch sizes, between 8K and 32K tokens.

- **Vocabulary and Tokenization for Multilingual Models**:
  - Multilingual tokenization often faces data imbalance; sampling too many sentences from high-resource languages (like English) can bias the vocabulary.
  - To counter this, models divide training data into subcorpora by language, rebalancing tokenization probabilities to upweight underrepresented languages, improving multilingual performance.

- **Multilingual Model Advantages and Challenges**:
  - Multilingual models offer practical advantages over creating many monolingual models and improve low-resource language performance by leveraging data from related, high-resource languages.
  - However, with many languages, these models suffer from a `curse of multilinguality`, where performance degrades per language, and a `linguistic accent` where high-resource language structures (often English) influence lower-resource languages.

## Contextual Embeddings
- Pretrained language models generate **contextual embeddings** for each token in a sentence, capturing its meaning within that context.
- For a given token $x_i$, its **contextual meaning** can be represented by the output vector $𝐳_i$ from the model's final layer, or by averaging the vectors from the last four layers.
  
  ![contextual embedding](./images/mlm/con.png)

- Unlike static embeddings like **word2vec**, which represent a `word type` with one vector, contextual embeddings provide distinct vectors for each `instance of a word` in its unique context.
- Contextual embeddings are versatile for tasks needing **context-aware word meaning**, such as measuring semantic similarity between words in specific contexts.

### Contextual Embeddings and Word Sense
- **Polysemy** means a word has multiple meanings 
  - 🍎 "mouse" as an animal or device, "bank" as an institution or river slope, 
  - Each distinct meaning is called a **sense**.
- Word senses, like **mouse₁** and **mouse₂**, can be found in resources like **WordNet**, which lists meanings in various languages.
  - Context generally clarifies the intended meaning.
- **Contextual embeddings** from models like BERT can visualize meaning differences geometrically, grouping instances of polysemous words based on context in high-dimensional space, rather than discrete lists.

  ![a BERT contextual embedding for the word die](./images/mlm/polysemy.png)

### Word Sense Disambiguation (WSD)
- **WSD** determines the correct meaning of a word based on context and a predefined sense inventory, such as WordNet, aiding text analysis and model interpretability.

  ![All-words WSD task](./images/mlm/wsdtask.png)

- **One sense per discourse rule** suggests that a word usually retains the same meaning within a given discourse, improving WSD consistency.
- The top-performing WSD method is a **[1-nearest-neighbor algorithm](https://paperswithcode.com/paper/deep-contextualized-word-representations)** that uses contextual embeddings, 
  
  ![The nearest-neighbor algorithm for WSD](./images/mlm/1nn.png)

  - where each word sense is represented by an averaged contextual embedding from labeled datasets; 
  - test instances are matched to the sense with the highest cosine similarity.

### Contextual Embeddings and Word Similarity
- **Word similarity** can be measured geometrically using `cosine similarity` between contextual embeddings, which indicates how close the meanings of two words are in context.
- **Anisotropy in embeddings** occurs when vectors from contextual embeddings tend to point in similar directions, resulting in high cosine similarities even between unrelated words.
- **Cosine similarity challenges** in anisotropic embeddings mean that the similarity measure can be influenced by `high-magnitude "rogue dimensions"` within the vector space.
- **Standardization (z-scoring)** helps reduce anisotropy by centering and scaling vectors: each vector’s mean is subtracted, and it is divided by its standard deviation across a corpus, improving isotropy.
- **Limitations** of cosine similarity remain, particularly its underestimation of similarity for high-frequency words, and further adjustments may be necessary for accurate meaning representation in NLP tasks.

## Fine-Tuning for Classification
- Pretrained language models leverage generalizations from vast text corpora, which can be applied to various tasks, either by **prompting the model in natural language** to guide its responses or outputs.
- For more **application-specific tasks**, models can be finetuned by adding specialized layers (heads) on top of the pretrained model, using labeled data to train these layers while keeping the main model parameters mostly unchanged.

### Sequence Classification
- **Sequence classification** tasks (e.g., sentiment analysis, spam detection) assign a single label to a text sequence, 
  - with some tasks involving many categories, like topic classification.
- To represent the input sequence, BERT introduces a special token `[CLS]` at the start, whose final-layer output vector serves as the sequence representation for classification.
  
  ![Sequence classification with a bidirectional transformer encoder](./images/mlm/seqc.png)
  
- For sentiment classification, the `output of [CLS]` (denoted as $𝐡^{L}_{CLS}$) is mapped to class scores via a learned weight matrix $𝐖_C$, then passed through **softmax** to produce predictions:
  - $𝐲 = \text{softmax}(𝐡^{L}_{CLS} 𝐖_C)$
- Finetuning updates both the classification weights $𝐖_C$ and, with minimal changes, the pretrained model parameters, especially in the final transformer layers, using labeled data and cross-entropy loss.

### Sequence-Pair Classification
- **Pair classification tasks** include applications like paraphrase detection, logical entailment, and discourse coherence, evaluating relationships between two input sequences.
- During **finetuning**, labeled sentence pairs are input into the model, with the `[CLS]` token representing the pair’s combined meaning; sentence pairs are separated by a `[SEP]` token.
- The **Multi-Genre Natural Language Inference (MultiNLI) dataset** supports tasks like entailment classification, with sentence pairs labeled as **entails, contradicts,** or **neutral** based on their semantic relationship.
- Example classifications:
  - *Neutral*: “Jon walked back to the town to the smithy” vs. “Jon traveled back to his hometown.”
  - *Contradicts*: “Tourist Information offices can be very helpful” vs. “Tourist Information offices are never of any help.”
  - *Entails*: “I’m confused” vs. “Not all of it is very clear to me.”
- For **MultiNLI finetuning**, sentence pairs are encoded with `[CLS]` as the classifier input, training the head to recognize entailment classes based on labeled data.

## Fine-Tuning for Sequence Labelling: Named Entity Recognition
- `Named Entity Recognition (NER)` One of the most common sequence labeling tasks.

### Named Entities
- NER identifies proper names in text and assigns them tags, such as 
  - PER (person), LOC (location), ORG (organization), and GPE (geo-political entity).
- The definition of named entities extends to 
  - temporal expressions (like dates) and numerical expressions (like prices).
- 🍎

```
Citing high fuel prices, [ORG United Airlines] said [TIME Friday] it
has increased fares by [MONEY $6] per round trip on flights to some
cities also served by lower-cost carriers. [ORG American Airlines], a
unit of [ORG AMR Corp.], immediately matched the move, spokesman
[PER Tim Wagner] said. [ORG United], a unit of [ORG UAL Corp.],
said the increase took effect [TIME Thursday] and applies to most
routes where it competes against discount carriers, such as [LOC Chicago]
to [LOC Dallas] and [LOC Denver] to [LOC San Francisco].
```

- NER is critical for linking text to structured information sources, analyzing sentiment, and anonymizing text, though it faces challenges with entity segmentation and type ambiguity.
- 🍎
```
[PER Washington] was born into slavery on the farm of James Burroughs.
[ORG Washington] went up 2 games to 1 in the four-game series.
Blair arrived in [LOC Washington] for what may well be his last state visit.
In June, [GPE Washington] passed a primary seatbelt law.
```


### BIO Tagging
- `BIO` tagging is a `sequence labeling method` for NER
  - using B (beginning), I (inside), and O (outside) tags to indicate the boundaries and types of entities.
- Each named entity type has distinct B and I tags, leading to a total of $2n + 1$ tags
  - where $n$ is the number of entity types.
- This method simplifies representing NER as a sequence modeling task similar to `part-of-speech` tagging, allowing for straightforward label assignment.
- Variants of BIO tagging include IO tagging (which loses boundary information) and BIOES tagging (which adds end and span tags).
- 🍎

| Words         | IO Label | BIO Label | BIOES Label |
|---------------|----------|-----------|-------------|
| Jane          | I-PER    | B-PER     | B-PER       |
| Villanueva    | I-PER    | I-PER     | E-PER       |
| of            | O        | O         | O           |
| United        | I-ORG    | B-ORG     | B-ORG       |
| Airlines      | I-ORG    | I-ORG     | I-ORG       |
| Holding       | I-ORG    | I-ORG     | E-ORG       |
| discussed     | O        | O         | O           |
| the           | O        | O         | O           |
| Chicago       | I-LOC    | B-LOC     | S-LOC       |
| route         | O        | O         | O           |
| .             | O        | O         | O           |


### Sequence Labeling
- Sequence labeling involves passing output vectors for each token to a classifier that generates a softmax distribution over possible tags.
- A feedforward classifier learns weights $𝐖_𝐊$ of size $[d \times k]$, where $k$ is the number of tags, using an argmax approach for final output tags.

![Sequence labeling for NER with a bidirectional transformer encoder.](./images/mlm/ner.png)

- $𝐲_i = \text{softmax}(𝐡_i^L 𝐖_𝐊)$
- $𝐭_i = \text{argmax}_k(𝐲_i)$
  -  $𝐲_i$ is a vector of probabilities over tags, and $k$ indexes the tags

- Alternatively, a conditional random field (CRF) layer can be used to consider global tag-level transitions for improved label predictions.
- Training data for NER is typically annotated with BIO tags, which must align with tokenized inputs, such as subword tokens from WordPiece tokenization.



### Evaluating Named Entity Recognition
- NER systems are evaluated using recall, precision, and F1 measures, focusing on the ratio of correctly labeled entities.
- The F1 score difference between systems can be assessed using statistical tests like the paired bootstrap test or randomization test.
- In NER, the evaluation counts entities rather than individual words, which complicates the evaluation due to segmentation errors.
- The mismatch between training on word-level data and evaluating on entity-level data creates challenges in accurately assessing NER system performance.

-🏃 Practice from HuggingFace NLP
  - [Token classification](https://huggingface.co/learn/nlp-course/chapter7/2?fw=pt)